In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
import os
from dotenv import load_dotenv

# .env file ka path
load_dotenv("../.env")  

# key ko variable me store karo
key = os.getenv("GROQ_API_KEY")

# optional check
print(key is not None)  


True


In [3]:
from langchain_groq import ChatGroq


In [4]:

llm = ChatGroq(
    groq_api_key=key,
    model_name="llama-3.1-8b-instant", 
    temperature=0.5
)


In [5]:
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x110e57770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x111214590>, model_name='llama-3.1-8b-instant', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import CallbackManager
from langchain_core.callbacks.stdout import StdOutCallbackHandler
import PyPDF2


In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct": "correct snswer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct": "correct snswer",
    },
}

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choices questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [9]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE 
)

In [10]:
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()


In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [17]:
review_chain = quiz_evaluation_prompt | llm | StrOutputParser()

In [32]:
def generate_evaluate_chain(inputs: dict):
    quiz = quiz_chain.invoke({
        "text": inputs["text"],
        "number": inputs["number"],
        "subject": inputs["subject"],
        "tone": inputs["tone"],
        "response_json": inputs["response_json"]
    })

    review = review_chain.invoke({
        "quiz": quiz,
        "subject": inputs["subject"]
    })

    return {
        "quiz": quiz,
        "review": review
    }


In [33]:
file_path=r"/Users/jessicawadhwa/mcqgen/data.txt"

In [34]:
file_path

'/Users/jessicawadhwa/mcqgen/data.txt'

In [35]:
with open(file_path, 'r') as file:
    TEXT=file.read()

In [36]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[5][6] The synonym self-teaching computers was also used during this time period.[7][8]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[9] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[10] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[9] Other researchers who have studied human cognitive systems contri

In [37]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct snswer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct snswer"}}'

In [38]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [39]:

def generate_evaluate_chain(inputs: dict):
    quiz = quiz_chain.invoke({
        "text": inputs["text"],
        "number": inputs["number"],
        "subject": inputs["subject"],
        "tone": inputs["tone"],
        "response_json": inputs["response_json"]
    })

    review = review_chain.invoke({
        "quiz": quiz,
        "subject": inputs["subject"]
    })

    return {
        "quiz": quiz,
        "review": review
    }

callback_manager = CallbackManager([StdOutCallbackHandler()])
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3,
    callback_manager=callback_manager
)
response = generate_evaluate_chain({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
})
print(response["quiz"])
print(response["review"])


 

### RESPONSE_JSON
{"1": {"mcq": "Who is credited with coining the term 'machine learning' in 1959?", "options": {"a": "Arthur Samuel", "b": "Donald Hebb", "c": "Walter Pitts", "d": "Nils Nilsson"}, "correct": "a"}, 
"2": {"mcq": "What was the primary objective of the Cybertron machine developed by Raytheon Company in the early 1960s?", "options": {"a": "Pattern recognition", "b": "Speech synthesis", "c": "Natural language processing", "d": "Computer vision"}, "correct": "a"}, 
"3": {"mcq": "According to Tom M. Mitchell's definition, what is the primary goal of machine learning?", "options": {"a": "To improve performance at tasks based on experience", "b": "To recognize patterns in data", "c": "To classify objects into categories", "d": "To make decisions based on rules"}, "correct": "a"}, 
"4": {"mcq": "What type of machine learning algorithm focuses on making decisions based on previous, unknown time?", "options": {"a": "Supervised Learning", "b": "Unsupervised Learning", "c": "Reinfo

In [41]:
print("Generated Quiz:\n", response["quiz"])
print("\nQuiz Review:\n", response["review"])



Generated Quiz:
 ### RESPONSE_JSON
{"1": {"mcq": "Who is credited with coining the term 'machine learning' in 1959?", "options": {"a": "Arthur Samuel", "b": "Donald Hebb", "c": "Walter Pitts", "d": "Nils Nilsson"}, "correct": "a"}, 
"2": {"mcq": "What was the primary objective of the Cybertron machine developed by Raytheon Company in the early 1960s?", "options": {"a": "Pattern recognition", "b": "Speech synthesis", "c": "Natural language processing", "d": "Computer vision"}, "correct": "a"}, 
"3": {"mcq": "According to Tom M. Mitchell's definition, what is the primary goal of machine learning?", "options": {"a": "To improve performance at tasks based on experience", "b": "To recognize patterns in data", "c": "To classify objects into categories", "d": "To make decisions based on rules"}, "correct": "a"}, 
"4": {"mcq": "What type of machine learning algorithm focuses on making decisions based on previous, unknown time?", "options": {"a": "Supervised Learning", "b": "Unsupervised Learni

In [44]:
response

{'quiz': '### RESPONSE_JSON\n{"1": {"mcq": "Who is credited with coining the term \'machine learning\' in 1959?", "options": {"a": "Arthur Samuel", "b": "Donald Hebb", "c": "Walter Pitts", "d": "Nils Nilsson"}, "correct": "a"}, \n"2": {"mcq": "What was the primary objective of the Cybertron machine developed by Raytheon Company in the early 1960s?", "options": {"a": "Pattern recognition", "b": "Speech synthesis", "c": "Natural language processing", "d": "Computer vision"}, "correct": "a"}, \n"3": {"mcq": "According to Tom M. Mitchell\'s definition, what is the primary goal of machine learning?", "options": {"a": "To improve performance at tasks based on experience", "b": "To recognize patterns in data", "c": "To classify objects into categories", "d": "To make decisions based on rules"}, "correct": "a"}, \n"4": {"mcq": "What type of machine learning algorithm focuses on making decisions based on previous, unknown time?", "options": {"a": "Supervised Learning", "b": "Unsupervised Learni

In [52]:
quiz = response.get("quiz")  # get the value, not the string "quiz"

# agar wo JSON string hai:
try:
    quiz_json = json.loads(quiz)
    print(quiz_json)
except json.JSONDecodeError:
    print("quiz is not a JSON string, printing as is:")
    print(quiz)



quiz is not a JSON string, printing as is:
### RESPONSE_JSON
{"1": {"mcq": "Who is credited with coining the term 'machine learning' in 1959?", "options": {"a": "Arthur Samuel", "b": "Donald Hebb", "c": "Walter Pitts", "d": "Nils Nilsson"}, "correct": "a"}, 
"2": {"mcq": "What was the primary objective of the Cybertron machine developed by Raytheon Company in the early 1960s?", "options": {"a": "Pattern recognition", "b": "Speech synthesis", "c": "Natural language processing", "d": "Computer vision"}, "correct": "a"}, 
"3": {"mcq": "According to Tom M. Mitchell's definition, what is the primary goal of machine learning?", "options": {"a": "To improve performance at tasks based on experience", "b": "To recognize patterns in data", "c": "To classify objects into categories", "d": "To make decisions based on rules"}, "correct": "a"}, 
"4": {"mcq": "What type of machine learning algorithm focuses on making decisions based on previous, unknown time?", "options": {"a": "Supervised Learning",

In [64]:
import pandas as pd
import json
import ast
from IPython.display import display

quiz_str = response.get("quiz")

# Convert stringified dict safely
try:
    quiz_dict = ast.literal_eval(quiz_str)
except (ValueError, SyntaxError):
    quiz_dict = None

# Build table
quiz_table_data = []
if quiz_dict and isinstance(quiz_dict, dict):
    for key, value in quiz_dict.items():
        mcq_text = value.get("mcq", "")
        options_text = " | ".join([f"{opt}: {val}" for opt, val in value.get("options", {}).items()])
        correct_text = value.get("correct", "")
        quiz_table_data.append({"MCQ": mcq_text, "Options": options_text, "Correct": correct_text})
else:
    quiz_table_data.append({"MCQ": quiz_str, "Options": "", "Correct": ""})

quiz=quiz_df = pd.DataFrame(quiz_table_data)

# Show full table
pd.set_option("display.max_colwidth", None)

# ✅ Correct way to display table in notebook
display(quiz_df)









,MCQ,Options,Correct
0,Who is credited with coining the term 'machine learning' in 1959?,a: Arthur Samuel | b: Donald Hebb | c: Walter Pitts | d: Nils Nilsson,a
1,What was the primary objective of the Cybertron machine developed by Raytheon Company in the early 1960s?,a: Pattern recognition | b: Speech synthesis | c: Natural language processing | d: Computer vision,a
2,"According to Tom M. Mitchell's definition, what is the primary goal of machine learning?",a: To improve performance at tasks based on experience | b: To recognize patterns in data | c: To classify objects into categories | d: To make decisions based on rules,a
3,"What type of machine learning algorithm focuses on making decisions based on previous, unknown time?",a: Supervised Learning | b: Unsupervised Learning | c: Reinforcement Learning | d: Deep Learning,c
4,"In 2016, which AI system used reinforcement learning techniques to defeat top human players?",a: AlphaGo | b: DeepMind | c: Google Assistant | d: IBM Watson,a


In [65]:
quiz.to_csv("machinelearning.csv", index=False)

,MCQ,Options,Correct
0,"### RESPONSE_JSON\n{""1"": {""mcq"": ""Who is credi...",,
